In [4]:
import pandas as pd
import os
import numpy as np
import re

dataDirectory = '/Users/jetfoiler/desktop/JFS'

df = pd.DataFrame(columns=['File Path','StartTime'])
df1 = pd.DataFrame(columns=['CAN ID','Parent Name','Signal Name','Offset','Length','Endianess','Type','Unit','Slope','Intercept'])
keywords = ['/"','name','offset','length','endianess','value','type','slope','unit', 'intercept']
count = 0
for file in os.listdir(dataDirectory):
    if file.endswith(".jfl"):
        count = count+1
        filePath = (os.path.join(dataDirectory, file))
        df.loc[count,['File Path']] = filePath
        sepType = '\t'
        json = 1
print(df)
#Code Block for extracting JSON information from header file of a .JFL 
#IMPORTANT ***ONLY EXTRACTS THE HEADER FILE FROM THE FIRST UPLOADED FILE
cnt = 0
result = ''
for i in range(0, len(df)):
    sigNameCount = 0
    fileToRead = (df.iloc[i]['File Path'])
    if json and i == 0:
        #Extract JSON header info
        with open(fileToRead, "r") as f:
            searchlines = f.readlines()
            for i, line in enumerate(searchlines):
                if "0x" in line:
                    jsonHeader = line
                    result = re.split(r"}]},{",jsonHeader)
            for i in range(0, len(result)):
                canIdSearch = re.search(r"0x...",result[i]).group(0)
                unparsedName = result[i].split(canIdSearch)[-1].split(keywords[0])[0]
                parseName = unparsedName[10:30]
                if parseName[0] == ':':
                    parseName = unparsedName[23:46]
                name = parseName
                canIdName = re.split('[^a-zA-Z]', name) 
                signalSplit = re.split("[^.\w]", unparsedName) 
                sigName = 0
                offset = 0
                length = 0
                endianess = ''
                value = 0
                _type = ''
                slope = 0
                unit = ''
                intercept = 0
                for varX in range(15,len(signalSplit)):
                    buff = signalSplit[varX]
                    if buff == keywords[1]:
                        sigName = signalSplit[varX + 3]
                        sigNameCount = sigNameCount + 1
                        df1.loc[sigNameCount,['CAN ID']] = canIdSearch
                        df1.loc[sigNameCount,['Parent Name']] = canIdName[0]
                        df1.loc[sigNameCount,['Signal Name']] = sigName
                    elif buff == keywords[2]: #offset
                        offset = signalSplit[varX + 3]
                        df1.loc[sigNameCount,['Offset']] = offset
                    elif buff == keywords[3]: #length
                        length = signalSplit[varX + 3]
                        df1.loc[sigNameCount,['Length']] = length
                    elif buff == keywords[4]: #endianess
                        endianess = signalSplit[varX + 3]
                        df1.loc[sigNameCount,['Endianess']] = endianess
                    elif buff == keywords[5]: #value
                        value = signalSplit[varX + 3]
                    elif buff == keywords[6]: #type
                        _type = signalSplit[varX + 3]
                        df1.loc[sigNameCount,['Type']] = _type
                    elif buff == keywords[7]: #slope
                        slope = signalSplit[varX + 3]
                        df1.loc[sigNameCount,['Slope']] = slope
                    elif buff == keywords[8]: #unit
                        unit = signalSplit[varX + 3]
                        df1.loc[sigNameCount,['Unit']] = unit
                    elif buff == keywords[9]: #intercept
                        intercept = signalSplit[varX + 4]
                        df1.loc[sigNameCount,['Intercept']] = intercept

# for whole dataframe
df1['Offset'] = df1['Offset'].replace(np.nan, 0)
df1['Length'] = df1['Length'].replace(np.nan, 1)
df1['Endianess'] = df1['Endianess'].replace(np.nan, '0')
df1['Type'] = df1['Type'].replace(np.nan, 0)
df1['Unit'] = df1['Unit'].replace(np.nan, 'U')
df1['Slope'] = df1['Slope'].replace(np.nan, 0)
df1['Intercept'] = df1['Intercept'].replace(np.nan, 0)



#df1 = df1.replace(np.nan, 0)
pd.options.display.max_rows = 999
df1

                                           File Path StartTime
1  /Users/jetfoiler/desktop/JFS/20190719-031005-0...       NaN
2  /Users/jetfoiler/desktop/JFS/20190720-031935-0...       NaN
3  /Users/jetfoiler/desktop/JFS/20190721-025724-0...       NaN
4  /Users/jetfoiler/desktop/JFS/20190721-025756-0...       NaN
5  /Users/jetfoiler/desktop/JFS/20190721-033608-a...       NaN
6  /Users/jetfoiler/desktop/JFS/20190721-034203-0...       NaN
7  /Users/jetfoiler/desktop/JFS/20190722-083451-a...       NaN
8  /Users/jetfoiler/desktop/JFS/20190722-084400-0...       NaN
9  /Users/jetfoiler/desktop/JFS/20190722-084444-0...       NaN


,CAN ID,Parent Name,Signal Name,Offset,Length,Endianess,Type,Unit,Slope,Intercept
1,0x106,NodeHealth,node_health_good,0,1,0,0,U,0,0
2,0x106,NodeHealth,can_voltage,8,16,little,0,V,0.01,
3,0x106,NodeHealth,board_voltage,24,16,little,0,V,0.01,
4,0x106,NodeHealth,board_current,40,16,little,0,A,0.001,
5,0x106,NodeHealth,board_temperature,56,8,0,0,C,1,20
6,0x110,BoardStatus,battery_discharge_ok,0,1,0,0,U,0,0
7,0x110,BoardStatus,throttle_derate,1,1,0,0,U,0,0
8,0x110,BoardStatus,water_detected,2,1,0,0,U,0,0
9,0x110,BoardStatus,ride_mode,8,8,0,0,U,0,0
10,0x110,BoardStatus,water_sensor_avg,16,8,0,0,U,0,0


In [5]:
def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val

def jfl_can_id(canId):
    buffId1 = ''
    bf2 = ''
    cJflId = re.split('x',canId)
    buffId = cJflId[1]
    bf2 =' '+buffId.lower()+' '
    return bf2

In [6]:
#Search for single CAN Id
cId1 = '0x490'
bfId1 = jfl_can_id(cId1)
msgNum1 = bfId1
cnId1 = cId1

#print(msgNum1,msgNum2,cId1,cId2)
rwMatch1 = df1.loc[df1['CAN ID']==cnId1].index.values
desiredParentName1 = df1.iloc[rwMatch1[0],1]
rowMatches1 = df1.loc[df1['Parent Name']==desiredParentName1].index.values
not4Bits = False
lessThan4Bits = False
k = 0

for k in range(0, len(df)):
    fileToRead = (df.iloc[k]['File Path'])
    msgType = "C"
    with open(fileToRead, "r") as b:
        searchlines = b.readlines()
        count = 0
    for i, line in enumerate(searchlines):
        if "C " and (msgNum1) in line:
            #print(line)
            if msgNum1 in line:
                #print(line)
                count = count + 1
                parseData = line.split(" ")
                mType = parseData[0]
                time = parseData[1]
                msgID = parseData[2]
                canID = '0x'+msgID  
                msgContent = parseData[3]
                c = int(msgContent,16)
                str(c)
                #print(msgContent)
                #print (" As binary: {:0>64s}".format(bin(c)[2:]))  
                for j in range (rowMatches1[0]-1,rowMatches1[0]+len(rowMatches1)-1):
                    columnIndex = (j+len(rowMatches1)) - rowMatches1[0+len(rowMatches1)-1] + k
                    lessThan4Bits = False
                    not4Bits = False
                    canId = df1.iloc[j,0]
                    parentName = df1.iloc[j,1]
                    signalName = df1.iloc[j,2]
                    
                    length = int(df1.iloc[j,4])
                    #print("Length%4:",length%4)
                    if length%4 != 0:
                        length = length
                        lessThan4Bits = True
                    else:
                        length = length / 4
                        lessThan4Bits = False
                        
                    offset = int(df1.iloc[j,3])
                    if offset%4 != 0 or lessThan4Bits == True:
                        not4Bits = True
                        for q in range (0,4):
                            lwrNum = j - q
                            offset = int(df1.iloc[lwrNum,3])
                            if offset%4 == 0:
                                offset = offset/4
                                break
                    else:
                        offset = offset/4
                        not4bits = False
                    #print("Offset:",offset,"Length:",length)
                    endianess = df1.iloc[j,5]
                    _type = df1.iloc[j,6]
                    unit = df1.iloc[j,7]
                    slope = df1.iloc[j,8]
                    if slope == '1e':
                        scalar = 100000
                    elif slope == '0.1':
                        scalar = 10
                    elif slope == '0.01':
                        scalar = 100
                    elif slope == '0.001':
                        scalar = 1000
                    elif slope == '0.0001':
                        scalar = 10000
                    else:
                        scalar = 1
                    intercept = df1.iloc[j,9]     
                    ofst = int(offset)
                    lngth = int(length)
                    buff = msgContent[ofst:ofst+lngth]
                       
                    #print('Offset:',ofst,'Length:',lngth,'Buffer:',buff,'Msg:',msgContent[ofst:ofst+lngth])
                    buffStr = ''
                    buffLilE = []
                    #print(signalName,'Offset: ', offset, 'Length:', length, buff,buffDec)
                    #endianess = 'little'
                    #determine byte read order based on endianness   
                    if canID == ('0x116'):
                        endianess = 'little'
                    if endianess == 'little':
                        
                        if lngth * 4 == 8:
                            buffLilE = buff[0]+buff[1]
                        elif lngth*4 == 16:
                            buffLilE = buff[2]+buff[3]+buff[0]+buff[1]
                        elif lngth*4 == 32:
                            buffLilE = buff[6]+buff[7]+buff[4]+buff[5]+buff[2]+buff[3]+buff[0]+buff[1]
                            
                        buffLength = len(buff)
                        buffStr = '0x'+buffLilE
                    else:
                        buffStr = '0x'+buff
                        
                    if not4Bits == True or lessThan4Bits == True:
                        bitLocation = int(df1.iloc[j,3])
                        bitNum = int(bitLocation - (offset*4))
                        buffBit = bin(int(buffStr, 16))[2:].zfill(4)
                        #print("4bits:",buffBit,"bitNum:",bitNum,"buffBit:",buffBit[bitNum])
                        buffDec = buffBit[bitNum]
                        #current from BMS is 2s complemented
                    else:
                        if _type == 'signed': 
                            #print('2s Complemented',buffStr)
                            out = twos_comp(int(buffStr,16), lngth*4)/scalar
                            buffDec = out
                        else:
                            #print(buffStr)
                            buffDec = int(buffStr,16)/scalar
                        
                    #how to handle big jumps in time with no data? Just put data gathered near eachother together?
                    #if ('serial_num' or 'firmware_version') in signalName:# or signalName == 'flux_y' or signalName == 'flux_z') and (int(time) >= 8461133 and int(time) <= 8481731):
                    print("Time:",time,  signalName,'Hex:',buffStr,'Dec:',buffDec)#, 'Row:', count,'Column:',columnIndex, 'File Num:', k)


Time: 2212017 repo Hex: 0x00 Dec: 0.0
Time: 2212017 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 2212017 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 2212017 changed_files Hex: 0x00 Dec: 0.0
Time: 2221811 repo Hex: 0x01 Dec: 1.0
Time: 2221811 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 2221811 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 2221811 changed_files Hex: 0x01 Dec: 1.0
Time: 2222103 repo Hex: 0x01 Dec: 1.0
Time: 2222103 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 2222103 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 2222103 changed_files Hex: 0x01 Dec: 1.0
Time: 2224167 repo Hex: 0x00 Dec: 0.0
Time: 2224167 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 2224167 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 2224167 changed_files Hex: 0x00 Dec: 0.0
Time: 2226207 repo Hex: 0x01 Dec: 1.0
Time: 2226207 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 2226207 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 2226207 changed_files Hex: 0x01 Dec: 1.0
Time: 2228271 repo Hex: 0

Time: 630209 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 630209 changed_files Hex: 0x01 Dec: 1.0
Time: 632233 repo Hex: 0x00 Dec: 0.0
Time: 632233 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 632233 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 632233 changed_files Hex: 0x00 Dec: 0.0
Time: 634243 repo Hex: 0x01 Dec: 1.0
Time: 634243 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 634243 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 634243 changed_files Hex: 0x01 Dec: 1.0
Time: 636246 repo Hex: 0x00 Dec: 0.0
Time: 636246 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 636246 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 636246 changed_files Hex: 0x00 Dec: 0.0
Time: 638269 repo Hex: 0x01 Dec: 1.0
Time: 638269 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 638269 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 638269 changed_files Hex: 0x01 Dec: 1.0
Time: 640289 repo Hex: 0x00 Dec: 0.0
Time: 640289 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 640289 build_time Hex: 0xa5e5da5c Dec: 

Time: 792346 repo Hex: 0x01 Dec: 1.0
Time: 792346 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 792346 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 792346 changed_files Hex: 0x01 Dec: 1.0
Time: 794347 repo Hex: 0x00 Dec: 0.0
Time: 794347 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 794347 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 794347 changed_files Hex: 0x00 Dec: 0.0
Time: 796397 repo Hex: 0x01 Dec: 1.0
Time: 796397 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 796397 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 796397 changed_files Hex: 0x01 Dec: 1.0
Time: 798467 repo Hex: 0x00 Dec: 0.0
Time: 798467 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 798467 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 798467 changed_files Hex: 0x00 Dec: 0.0
Time: 800489 repo Hex: 0x01 Dec: 1.0
Time: 800489 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 800489 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 800489 changed_files Hex: 0x01 Dec: 1.0
Time: 802490 repo Hex: 0x00 Dec: 0.0
Time: 80

Time: 992934 repo Hex: 0x00 Dec: 0.0
Time: 992934 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 992934 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 992934 changed_files Hex: 0x00 Dec: 0.0
Time: 994978 repo Hex: 0x01 Dec: 1.0
Time: 994978 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 994978 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 994978 changed_files Hex: 0x01 Dec: 1.0
Time: 997003 repo Hex: 0x00 Dec: 0.0
Time: 997003 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 997003 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 997003 changed_files Hex: 0x00 Dec: 0.0
Time: 999023 repo Hex: 0x01 Dec: 1.0
Time: 999023 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 999023 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 999023 changed_files Hex: 0x01 Dec: 1.0
Time: 1001026 repo Hex: 0x00 Dec: 0.0
Time: 1001026 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 1001026 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1001026 changed_files Hex: 0x00 Dec: 0.0
Time: 1003047 repo Hex: 0x01 Dec: 1.0
Tim

Time: 1175241 repo Hex: 0x00 Dec: 0.0
Time: 1175241 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 1175241 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1175241 changed_files Hex: 0x00 Dec: 0.0
Time: 1177264 repo Hex: 0x01 Dec: 1.0
Time: 1177264 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 1177264 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1177264 changed_files Hex: 0x01 Dec: 1.0
Time: 1179271 repo Hex: 0x00 Dec: 0.0
Time: 1179271 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 1179271 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1179271 changed_files Hex: 0x00 Dec: 0.0
Time: 1181290 repo Hex: 0x01 Dec: 1.0
Time: 1181290 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 1181290 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1181290 changed_files Hex: 0x01 Dec: 1.0
Time: 1183302 repo Hex: 0x00 Dec: 0.0
Time: 1183302 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 1183302 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1183302 changed_files Hex: 0x00 Dec: 0.0
Time: 1185322 repo Hex: 0

...

Time: 773256 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 773256 changed_files Hex: 0x00 Dec: 0.0
Time: 775319 repo Hex: 0x01 Dec: 1.0
Time: 775319 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 775319 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 775319 changed_files Hex: 0x01 Dec: 1.0
Time: 777330 repo Hex: 0x00 Dec: 0.0
Time: 777330 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 777330 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 777330 changed_files Hex: 0x00 Dec: 0.0
Time: 779380 repo Hex: 0x01 Dec: 1.0
Time: 779380 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 779380 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 779380 changed_files Hex: 0x01 Dec: 1.0
Time: 781393 repo Hex: 0x00 Dec: 0.0
Time: 781393 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 781393 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 781393 changed_files Hex: 0x00 Dec: 0.0
Time: 783397 repo Hex: 0x01 Dec: 1.0
Time: 783397 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 783397 build_time Hex: 0xa5e5da5c Dec: 

Time: 961619 repo Hex: 0x01 Dec: 1.0
Time: 961619 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 961619 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 961619 changed_files Hex: 0x01 Dec: 1.0
Time: 963620 repo Hex: 0x00 Dec: 0.0
Time: 963620 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 963620 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 963620 changed_files Hex: 0x00 Dec: 0.0
Time: 965628 repo Hex: 0x01 Dec: 1.0
Time: 965628 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 965628 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 965628 changed_files Hex: 0x01 Dec: 1.0
Time: 967630 repo Hex: 0x00 Dec: 0.0
Time: 967630 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 967630 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 967630 changed_files Hex: 0x00 Dec: 0.0
Time: 969636 repo Hex: 0x01 Dec: 1.0
Time: 969636 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 969636 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 969636 changed_files Hex: 0x01 Dec: 1.0
Time: 971661 repo Hex: 0x00 Dec: 0.0
Time: 97

Time: 1139133 repo Hex: 0x01 Dec: 1.0
Time: 1139133 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 1139133 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1139133 changed_files Hex: 0x01 Dec: 1.0
Time: 1141135 repo Hex: 0x00 Dec: 0.0
Time: 1141135 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 1141135 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1141135 changed_files Hex: 0x00 Dec: 0.0
Time: 1143152 repo Hex: 0x01 Dec: 1.0
Time: 1143152 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 1143152 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1143152 changed_files Hex: 0x01 Dec: 1.0
Time: 1145155 repo Hex: 0x00 Dec: 0.0
Time: 1145155 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 1145155 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1145155 changed_files Hex: 0x00 Dec: 0.0
Time: 1147165 repo Hex: 0x01 Dec: 1.0
Time: 1147165 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 1147165 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1147165 changed_files Hex: 0x01 Dec: 1.0
Time: 1149170 repo Hex: 0

Time: 1328855 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 1328855 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1328855 changed_files Hex: 0x01 Dec: 1.0
Time: 1330862 repo Hex: 0x00 Dec: 0.0
Time: 1330862 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 1330862 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1330862 changed_files Hex: 0x00 Dec: 0.0
Time: 1332940 repo Hex: 0x01 Dec: 1.0
Time: 1332940 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 1332940 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1332940 changed_files Hex: 0x01 Dec: 1.0
Time: 1334949 repo Hex: 0x00 Dec: 0.0
Time: 1334949 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 1334949 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1334949 changed_files Hex: 0x00 Dec: 0.0
Time: 1336956 repo Hex: 0x01 Dec: 1.0
Time: 1336956 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 1336956 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1336956 changed_files Hex: 0x01 Dec: 1.0
Time: 1338982 repo Hex: 0x00 Dec: 0.0
Time: 1338982 commit_hash

Time: 1520774 changed_files Hex: 0x00 Dec: 0.0
Time: 1524811 repo Hex: 0x00 Dec: 0.0
Time: 1524811 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 1524811 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1524811 changed_files Hex: 0x00 Dec: 0.0
Time: 1526816 repo Hex: 0x01 Dec: 1.0
Time: 1526816 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 1526816 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1526816 changed_files Hex: 0x01 Dec: 1.0
Time: 1528822 repo Hex: 0x00 Dec: 0.0
Time: 1528822 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 1528822 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1528822 changed_files Hex: 0x00 Dec: 0.0
Time: 1530840 repo Hex: 0x01 Dec: 1.0
Time: 1530840 commit_hash Hex: 0x8656 Dec: 34390.0
Time: 1530840 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1530840 changed_files Hex: 0x01 Dec: 1.0
Time: 1532854 repo Hex: 0x00 Dec: 0.0
Time: 1532854 commit_hash Hex: 0x2922 Dec: 10530.0
Time: 1532854 build_time Hex: 0xa5e5da5c Dec: 2783304284.0
Time: 1532854 changed_fil